<a href="https://colab.research.google.com/github/luislizcano/seagrass-mapping-tb/blob/main/Postprocessing/01_Mosaics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Annual seagrass mosaics**

Script to create annual seagrass mosaics by applying conditions to produce the best output possible.

Script by: Luis Lizcano-Sandoval (2021)<br/>
College of Marine Sciences, University of South Florida<br/>
luislizcanos@usf.edu / lsandoval@sig-gis.com<br/>
Updated: 12/26/2024

**Cite as:** *Lizcano-Sandoval, L.; Beck, M.W.; Scolaro, S.; Sherwood, E.T.; Muller-Karger, F. (2025) Cloud-Based Satellite Remote Sensing for Enhancing Seagrass Monitoring and Ecosystem Management (In preparation)*

### **Import packages**

In [ ]:
## Authenticate your EE account
import ee
ee.Authenticate()
ee.Initialize(project='earth-engine-252816')

In [ ]:
## Clone github repo:
!git clone https://github.com/luislizcano/seagrass-mapping-tb.git

In [ ]:
## Run this cell to mount your Google Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.insert(0,'/content/seagrass-mapping-tb')
sys.path.append('/content/seagrass-mapping-tb/Postprocessing/')

In [ ]:
## Verify you loaded the EE module correctly:
import geemap
import functions
print('EE version: ',ee.__version__)

EE version:  1.4.3


### **Load classified images**


In [ ]:
## Define segment
segmentCode = 'LTB'
year = '2023'
frequency = 10 ## It can't be higher than the number of images available
smoother = 1 ## 0 or 1, use 0 to do not apply smoother to the mosaic

In [ ]:
## Import imageCollection containing classified images
classified = ee.ImageCollection('users/lizcanosandoval/TB_seagrass_data/classified/TB'+year)

## Filter collection
segmentCollection = classified.filter(ee.Filter.eq('name_code', segmentCode))\
                              .filter(ee.Filter.eq('year', year))
## Import TB segments
segment = ee.FeatureCollection("users/lizcanosandoval/Seagrass/TBEP_Segments_Adapted")\
                .filter(ee.Filter.eq('name_code', segmentCode))
print('Images available: ',segmentCollection.size().getInfo())

Images available:  12


### **Mosaicking**

In [ ]:
## Select seagrass pixels only
## Output will have only seagrass pixels with value = 1
maskCollection = segmentCollection.map(functions.seagrassMask)

## Sum seagrass maps (mosaic)
## Pixel values of the output are considered as frequency of seagrass presence
imgSum = maskCollection.sum().clip(segment)

## Apply smoother to stack of images
if smoother == 1:
  imgSum = imgSum.convolve(ee.Kernel.euclidean(**{
      'radius':1,  # Edit to increase smoothing window
      'units':'pixels',
      'normalize':True}))

## Generate final mosaic with frequency condition
finalMosaic = functions.mosaic(imgSum, frequency)

In [ ]:
## Visualize image
values = imgSum.rename('b1').reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': segment,
    'scale': 10})
vis_params = {
    'min': values.get('b1_min').getInfo(),
    'max': values.get('b1_max').getInfo(),
    'palette': ['#fff0f0','#ffc8c8','#ff8989','#ff4444','#dd0000','#880000']
}
Map = geemap.Map()
Map.centerObject(segment, 12)
Map.addLayer(imgSum, vis_params, 'Seagrass frequency')
Map.addLayer(finalMosaic, {'palette':'green'}, 'Final Mosaic')
Map

### Calculate area (km2)

In [ ]:
area = getArea(finalMosaic.eq(1), segment)
print('Seagrass Area (km2): ', area)

### **Export image to Assets:**

Export to Drive if going to edit the mosaic using QGIS. If not, export to Assets if want to save it in your EE account.

In [ ]:
tileid = segmentCollection.first().get('tile_id').getInfo()
fileName = tileid+'_'+year+'_'+segmentCode

task = ee.batch.Export.image.toDrive(
  image= finalMosaic.toInt32(),
  description= fileName,
  region= segment,
  crs= 'EPSG:4326',
  scale= 10,
  maxPixels= 1e13,
  fileFormat= 'GeoTIFF'
)
task.start()